# Inspect forecast

In [ ]:
import pandas as pd
# from darksky import forecast
import requests
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.palettes import Spectral6

In [ ]:
path_list = ['33.3442655,126.4327353',
             '33.5266085,126.3729323',
             '33.4459325,126.9488463']
wind = []
for i, path_ in enumerate(path_list):
    response = requests.get('https://darksky.net/forecast/'+path_+'/ca12/en.json')
    df = pd.read_json(response.text)
    hourly_= df.hourly['data']
    wind.append(pd.DataFrame(hourly_).windSpeed.values)
# print(wind)

In [ ]:
# Plotting
s = figure(plot_width=1400, plot_height=750, title=None)

x = range(1, len(wind[0])-1)
for i, wind_ in enumerate(wind):
    y = wind_[0:-2]
    s.circle(x, y, size=4, color=Spectral6[i], alpha=0.5)

show(s)

In [ ]:
# https://darksky.net/forecast/37.5667,126.9783/ca12/en.json
path1 = '37.5667,126.9783'
request=Request('https://darksky.net/forecast/'+path1+'/ca12/en.json')
response = urlopen(request)
elevations = response.read()

In [ ]:
df.index[0] + pd.DateOffset(years=1)

In [ ]:
df = pd.read_pickle('../data/pickles/df_48_l_res1h.pickle')
print(df)

In [ ]:
# Plotting
hover = plotting.create_hover_tool()
fig = plotting.create_asset_graph(df.y, forecasts=df[["yhat", "yhat_upper", "yhat_lower"]],
                                  y_label="MW",
                                  hover_tool=hover)
show(fig)

In [ ]:
# Plotting
x = df.index
try:
    y1 = df.actual
    y2 = df['95']
except:
    y1 = df.y
    y2 = df.yhat
    
s1 = figure(plot_width=1400, plot_height=750, title=None)
s1.circle(x, y1, size=4, color="navy", alpha=0.5)
s1.square(x, y2, size=4, color="orange", alpha=0.5)

show(s1)

# Try storing market data from csv into a pickle

In [ ]:
import pandas as pd
from forecasting import make_rolling_forecast
from models import Market
import models
from datetime import date

In [ ]:
markets=[]

In [ ]:
df = pd.read_csv('../data/German day-ahead prices 20140101-20160630.csv', index_col=0, parse_dates = True, names = {'y'})
ix = pd.DatetimeIndex(start=df.index[0], periods=len(df.index), freq='1H')
df = pd.DataFrame(data=df.y.values, index=ix, columns={'EPEX_DA'})

In [ ]:
market_type = models.market_types['day_ahead']
predictions = make_rolling_forecast(df.y, market_type)
print(predictions.head)

In [ ]:
res = "1D"
res_df = df.resample(res).mean()
market_col_name = 'EPEX_DA'

In [ ]:
market_df = pd.DataFrame(index=res_df.index, columns=["actual", "yhat", "yhat_upper", "yhat_lower"])
market_df.actual = res_df[market_col_name]
market = Market(name=market_col_name, market_type_name=market_type)

In [ ]:
for conf in ["yhat", "yhat_upper", "yhat_lower"]:
    market_df[conf] = predictions[conf]
# market_df.to_pickle("../data/pickles/df_%s_res%s.pickle" % (market.name, res))

# Sandbox testing

In [ ]:
df2 = pd.DataFrame({ 'day': pd.Series([date(2012, 1, 1), date(2012, 1, 3)]), 'b' : pd.Series([0.22, 0.3]) })
print(df2)
df2 = df2.set_index('day')
print(df2)
ix = pd.DatetimeIndex(start=date(2012, 1, 1), end=date(2012, 1, 31), freq='D')
print(ix)
df3 = df2.reindex(ix)
print(df3)